In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/Colab Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [ ]:
import csv
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import lightgbm as lgbm


import json

pd.set_option('display.max_columns', 145)
pd.set_option("display.max_rows", None)

from joblib import dump, load

## Play Tourney Functions

In [ ]:
clf = load('lolranklgbm.joblib')

In [ ]:
df = pd.read_csv('lol_global.csv', dtype={'finalteam1id':'str', 'finalteam2id':'str'})
df['gamedate'] = pd.to_datetime(df['gamedate'])
df.head()

,platformgameid,gamedate,leagueid,leaguename,tournamentname,finalteam1id,finalteam2id,winningteam,killdifferential_end20av1,killdifferential_end20av2,killdifferential_end90av1,killdifferential_end90av2,killdifferential_end365av1,killdifferential_end365av2,avgflashdiff_b4objectives20av1,avgflashdiff_b4objectives20av2,avgflashdiff_b4objectives90av1,avgflashdiff_b4objectives90av2,avgflashdiff_b4objectives365av1,avgflashdiff_b4objectives365av2,golddifferential_end20av1,golddifferential_end20av2,golddifferential_end90av1,golddifferential_end90av2,golddifferential_end365av1,golddifferential_end365av2,avggoldratio_b4objectives20av1,avggoldratio_b4objectives20av2,avggoldratio_b4objectives90av1,avggoldratio_b4objectives90av2,avggoldratio_b4objectives365av1,avggoldratio_b4objectives365av2,teamwithfirstblood20av1,teamwithfirstblood20av2,teamwithfirstblood90av1,teamwithfirstblood90av2,teamwithfirstblood365av1,teamwithfirstblood365av2,firstheraldteam20av1,firstheraldteam20av2,firstheraldteam90av1,firstheraldteam90av2,firstheraldteam365av1,firstheraldteam365av2,firstdragonteam20av1,firstdragonteam20av2,firstdragonteam90av1,firstdragonteam90av2,firstdragonteam365av1,firstdragonteam365av2,winningteam20av1,winningteam20av2,winningteam90av1,winningteam90av2,winningteam365av1,winningteam365av2,towerskilled_team120ctav1,towerskilled_team220ctav2,towerskilled_team190ctav1,towerskilled_team290ctav2,towerskilled_team1365ctav1,towerskilled_team2365ctav2,totaldeaths_team120ctav1,totaldeaths_team220ctav2,totaldeaths_team190ctav1,totaldeaths_team290ctav2,totaldeaths_team1365ctav1,totaldeaths_team2365ctav2,dragonskilled_team120ctav1,dragonskilled_team220ctav2,dragonskilled_team190ctav1,dragonskilled_team290ctav2,dragonskilled_team1365ctav1,dragonskilled_team2365ctav2,baronskilled_team120ctav1,baronskilled_team220ctav2,baronskilled_team190ctav1,baronskilled_team290ctav2,baronskilled_team1365ctav1,baronskilled_team2365ctav2,pex1,pex2,league1,league2,leagueinterwinrate1,leagueintergames1,leagueinterwinrate2,leagueintergames2,teaminterwinsdiff,teamintergamesdiff,vswinrate1,vswinrate2,vsgames,leaguevswinrate1,leaguevswinrate2,strength20_1,strength20_2,strength90_1,strength90_2,strength365_1,strength365_2
0,ESPORTSTMNT01:1370750,2020-05-03 18:14:27.102000+00:00,98767991332355509,cblol-brazil,cblol_2020_split1,100205576309502431,100205576307813373,0,0.026953,-0.015273,-0.107825,0.019200,-0.099843,0.051007,-0.285714,-0.042262,-0.272751,-0.099123,-0.233787,-0.085606,0.011121,0.001251,-0.015010,0.014538,-0.015028,0.020348,1.018246,1.009439,0.996460,1.017516,0.992137,1.027452,0.500000,0.428571,0.555556,0.611111,0.476190,0.619048,0.375000,0.428571,0.333333,0.555556,0.285714,0.619048,0.750000,0.571429,0.611111,0.500000,0.619048,0.523810,0.500000,0.714286,0.388889,0.722222,0.428571,0.714286,7.125000,7.500000,5.444444,7.421053,5.428571,7.136364,15.375000,15.250000,14.388889,15.157895,14.523810,14.772727,3.000000,3.000000,2.222222,3.000000,2.285714,3.136364,1.250000,0.375000,0.666667,0.736842,0.761905,0.772727,0.419876,0.662732,cblol-brazil,cblol-brazil,-1.0,-1.0,-1.0,-1.0,0,0,0.200000,0.800000,5.0,-1.0,-1.0,0.481673,0.641023,0.399286,0.606505,0.435534,0.615043
1,ESPORTSTMNT01:1370787,2020-05-04 09:29:20.636000+00:00,98767991349978712,ljl-japan,ljl_spring_2020,101428372605353526,100285330168091787,0,-1.000000,0.272500,0.013102,0.135587,0.013102,0.135587,-1.000000,0.343750,-0.013784,0.146591,-0.013784,0.146591,-1.000000,0.072896,0.008046,0.037940,0.008046,0.037940,-1.000000,1.094349,1.000167,1.054801,1.000167,1.054801,-1.000000,1.000000,0.526316,0.700000,0.526316,0.700000,-1.000000,1.000000,0.526316,0.600000,0.526316,0.600000,-1.000000,1.500000,0.736842,0.600000,0.736842,0.600000,-1.000000,1.500000,0.684211,0.700000,0.684211,0.700000,-1000.000000,7.500000,5.842105,6.181818,5.842105,6.181818,-1000.000000,8.250000,8.894737,9.500000,8.894737,9.500000,-1000.000000,3.000000,3.421053,2.545455,3.421053,2.545455,-1000.000000,0.750000,0.842105,0.727273,0.842105,0.727273,0.669438,

In [ ]:
#df.info(max_cols=200)

In [ ]:
#creates stats to input into model for later prediction
#give df only of times before tourney
def create_game_stats(df, team1, team2):
  gamedate = pd.to_datetime(df[['gamedate']].max().values[0], utc=True)

  drop_cols = ['winningteam','finalteam1id', 'finalteam2id','platformgameid', 'gamedate',	'leagueid',	'leaguename',	'tournamentname', 'league1', 'league2']
  other_cols = ['teaminterwinsdiff', 'teamintergamesdiff', 'vswinrate1', 'vswinrate2', 'vsgames', 'leaguevswinrate1','leaguevswinrate2', 'league1', 'league2']
  t1_cols = [col for col in df.columns if col[-1]=='1' and col not in other_cols]
  t2_cols = [col for col in df.columns if col[-1]=='2' and col not in other_cols]

  out = {col:0 for col in df.columns if col not in drop_cols}
  #out = {}

  if len(df)==0:
    return out

  df1 = df[(df['finalteam1id']==team1) | (df['finalteam2id']==team1)]
  #print('df1len: ',len(df1))

  if len(df1)!=0:
    id = df1[['gamedate']].idxmax().values[0]
  else:
    df1 = df
    id = df.index[0]
    league1 = df['league1'][id]

  if df1['finalteam1id'][id]==team1:
    league1 = df1['league1'][id]
    for col in t1_cols:
      out[col] = df1[col][id]
  if df1['finalteam2id'][id]==team1:
    league1 = df1['league2'][id]
    for i,col in enumerate(t2_cols):
      out[t1_cols[i]] = df1[col][id]

  df2 = df[(df['finalteam1id']==team2) | (df['finalteam2id']==team2)]

  if len(df2)!=0:
    id = df2[['gamedate']].idxmax().values[0]
  else:
    df2 = df
    id = df.index[0]
    league2 = df['league2'][id]

  if df2['finalteam1id'][id]==team2:
    league2 = df2['league1'][id]
    for i,col in enumerate(t1_cols):
      out[t2_cols[i]] = df2[col][id]
  if df2['finalteam2id'][id]==team2:
    league2 = df2['league2'][id]
    for col in t2_cols:
      out[col] = df2[col][id]

  interdf = df[(df['leaguename']=='worlds') | (df['leaguename']=='msi')]
  interdf1 = interdf[interdf['finalteam1id']==team1]
  interwins1 = sum([1 if winner==1 else 0 for winner in interdf1['winningteam']])
  intergames1 = len(interdf1)
  interdf1 = interdf[interdf['finalteam2id']==team1]
  interwins1 += sum([1 if winner==0 else 0 for winner in interdf1['winningteam']])
  intergames1 += len(interdf1)

  interdf2 = interdf[interdf['finalteam1id']==team2]
  interwins2 = sum([1 if winner==1 else 0 for winner in interdf2['winningteam']])
  intergames2 = len(interdf2)
  interdf2 = interdf[interdf['finalteam2id']==team2]
  interwins2 += sum([1 if winner==0 else 0 for winner in interdf2['winningteam']])
  intergames2 += len(interdf2)

  out['teaminterwinsdiff'] = interwins1-interwins2
  out['teamintergamesdiff'] = intergames1-intergames2


  vsdf1 = df[(df['finalteam1id']==team1) & (df['finalteam2id']==team2)]
  vsdf1 = vsdf1[(gamedate - vsdf1['gamedate'] < pd.Timedelta(365,"d"))]
  wins1 = sum([1 if winner==1 else 0 for winner in vsdf1['winningteam']])
  wins2 = sum([1 if winner==0 else 0 for winner in vsdf1['winningteam']])

  vsdf2 = df[(df['finalteam1id']==team2) & (df['finalteam2id']==team1)]
  vsdf2 = vsdf2[(gamedate - vsdf2['gamedate'] < pd.Timedelta(365,"d"))]
  wins1 += sum([1 if winner==0 else 0 for winner in vsdf2['winningteam']])
  wins2 += sum([1 if winner==1 else 0 for winner in vsdf2['winningteam']])

  if wins1+wins2==0:
    out['vswinrate1'] = -1
    out['vswinrate2'] = -1
    out['vsgames'] = -1
  else:
    out['vswinrate1'] = wins1/(wins1+wins2)
    out['vswinrate2'] = wins2/(wins1+wins2)
    out['vsgames'] = wins1+wins2


  leaguevsdf = interdf[(interdf['league1']==league1) & (interdf['league2']==league2)]
  wins1 = sum([1 if winner==1 else 0 for winner in leaguevsdf['winningteam']])
  wins2 = sum([1 if winner==0 else 0 for winner in leaguevsdf['winningteam']])

  leaguevsdf = interdf[(interdf['league1']==league2) & (interdf['league2']==league1)]
  wins1 = sum([1 if winner==0 else 0 for winner in leaguevsdf['winningteam']])
  wins2 = sum([1 if winner==1 else 0 for winner in leaguevsdf['winningteam']])

  if wins1+wins2==0:
    out['leaguevswinrate1'] = -1
    out['leaguevswinrate2'] = -1
  else:
    out['leaguevswinrate1'] = wins1/(wins1+wins2)
    out['leaguevswinrate2'] = wins2/(wins1+wins2)


  #scale
  out['strength365_1'] = out['strength365_1']*out['leagueintergames1']/(out['leagueintergames2']+0.001)
  out['strength90_1'] = out['strength90_1']*out['leagueintergames1']/(out['leagueintergames2']+0.001)
  out['strength20_1'] = out['strength20_1']*out['leagueintergames1']/(out['leagueintergames2']+0.001)

  out['strength365_2'] = out['strength365_2']*out['leagueintergames2']/(out['leagueintergames1']+0.001)
  out['strength90_2'] = out['strength90_2']*out['leagueintergames2']/(out['leagueintergames1']+0.001)
  out['strength20_2'] = out['strength20_2']*out['leagueintergames2']/(out['leagueintergames1']+0.001)

  #handle emptydf
  if len(df[(df['finalteam1id']==team1) | (df['finalteam2id']==team1)])==0:
    #print('nogames1')
    for col in t1_cols:
      out[col]=-1
    for col in other_cols:
      if col not in ['league1', 'league2']:
        out[col]=0
  if len(df[(df['finalteam1id']==team2) | (df['finalteam2id']==team2)])==0:
    #print('nogames2')
    for col in t2_cols:
      out[col]=-1
    for col in other_cols:
      if col not in ['league1','league2']:
        out[col]=0

  return out


In [ ]:
#tests
#team1 = '102235771678061291' #turkey wildcats
#team2 = '98767991877340524' #c9
#game = {col:[] for col in df.columns if col not in ['winningteam','finalteam1id', 'finalteam2id','platformgameid', 'gamedate', 'leagueid',	'leaguename',	'tournamentname', 'league1', 'league2']}
#print(len(game.keys()))
#d = create_game_stats(df, team1, team2)
#print('len d :',len(d.keys()))
#for col in d.keys():
#  game[col].append(d[col])
#print(game)
#clf.predict_proba(pd.DataFrame(game))[0]#[0]

In [ ]:
#predicts results of round robin tourney two games for each pair swap sides for second game
def play_tourney(teams, df, clf):

  n_teams = len(teams)
  drop_cols = ['winningteam','finalteam1id', 'finalteam2id','platformgameid', 'gamedate', 'leagueid',	'leaguename',	'tournamentname', 'league1', 'league2']
  wins = {team:0 for team in teams}

  for team1 in teams:
    for team2 in teams:
      if(team1!=team2):
        game1 = {col:[] for col in df.columns if col not in drop_cols}
        d1 = create_game_stats(df, team1, team2)
        for col in d1.keys():
          game1[col].append(d1[col])
        result1 = clf.predict(pd.DataFrame(game1))[0]
        if result1==1:
          wins[team1]+=1
        else:
          wins[team2]+=1

  return wins

In [ ]:
#tests

#kt,geng, dpkia, t1, kdf, 100t, immpro, c9, g2
#teams = ['99566404579461230', '100205573495116443', '100725845018863243', '98767991853197861', '99566404581868574', '99294153828264740', '98767991930907107', '98767991877340524', '98767991926151025', '98926509885559666', '102235771678061291']

#test
#teams = ['101428372605353526', '98767991853197861']

#lec
#teams = ['98767991926151025', '107563714667537640', '103461966965149786', '101383793572656373', '98767991866488695', '101383793567806688', '101383793569248484', '109637393694097670', '101383793574360315', '99322214695067838','98926509885559666']
#res = play_tourney(teams, df, clf)
#team_df = pd.read_json("esports-data/teams.json", dtype={'team_id':'str'})
mp = {team_df['team_id'][i]:team_df['name'][i] for i in team_df.index}
#team_df.head()
#print(res)
#print({mp[key]:res[key] for key in res.keys()})

In [ ]:
#create_game_stats(df,'101428372605353526', '98767991853197861')

In [ ]:
#for col in d.keys():
#  game[col].append(d[col])
#print(game)
#clf.predict_proba(pd.DataFrame(game))[0]#[0]

In [ ]:
#df.info()

##Play All Tourneys

In [ ]:
tourney_df = pd.read_json("esports-data/tournaments.json", dtype={'id':'str'})
tourney_df['startDate'] = pd.to_datetime(tourney_df['startDate'], utc=True)
tourney_df['endDate'] = pd.to_datetime(tourney_df['startDate'], utc=True)
tourney_df = tourney_df[tourney_df['endDate'].dt.year>=2022]
tourney_df = tourney_df.set_index('id')

tourney_df.head()

,leagueId,name,slug,sport,startDate,endDate,stages
id,,,,,,,
110733838935136200,109518549825754240,#2 Summer 2023,nacl_qualifiers_2_summer_2023,lol,2023-07-20 00:00:00+00:00,2023-07-20 00:00:00+00:00,"[{'name': 'Groups', 'type': None, 'slug': 'gro..."
110428723804419399,105266108767593296,Summer 2023,gll_summer_2023,lol,2023-06-04 00:00:00+00:00,2023-06-04 00:00:00+00:00,"[{'name': 'Regular Season', 'type': None, 'slu..."
110424377524465827,107407335299756368,Summer 2023,elite_series_summer_2023,lol,2023-06-04 00:00:00+00:00,2023-06-04 00:00:00+00:00,"[{'name': 'Regular Season', 'type': None, 'slu..."
110349992504762921,105266074488398656,Summer 2023,superliga_summer_2023,lol,2023-05-28 00:00:00+00:00,2023-05-28 00:00:00+00:00,"[{'name': 'Regular Season', 'type': None, 'slu..."
110310652404523974,105266088231437424,Summer 2023,ultraliga_summer_2023,lol,2023-05-28 00:00:00+00:00,2023-05-28 00:00:00+00:00,"[{'name': 'Regular Season', 'type': None, 'slu..."


In [ ]:
#get list of teams for tourney with tourney id = tourney_id
def get_teams(tourney_df, tourney_id):
  teams = []

  stages = tourney_df['stages'].loc[tourney_id]
  for stage in stages:
    for section in stage['sections']:
      for mat in section['matches']:
        for team in mat['teams']:
          teams.append(team['id'])

  teams = set(teams)
  return list(teams)

In [ ]:
#t = get_teams(tourney_df, '110733838935136200')
#t

In [ ]:
#play all tourneys save results

team_df = pd.read_json("esports-data/teams.json", dtype={'team_id':'str'})
mp = {team_df['team_id'][i]:team_df['name'][i] for i in team_df.index}

rankings = {}
for tourney_id in tourney_df.index:
  date = tourney_df['startDate'][tourney_id]
  teams = get_teams(tourney_df, tourney_id)
  #print(teams)
  prevdf = df[df['gamedate']<date]
  result = play_tourney(teams, prevdf, clf)
  ranking={}
  for key in result.keys():
    if key in mp.keys():
      ranking[key] = result[key]
    else:
      ranking[key] = result[key]
  rankings[tourney_id] = dict(sorted(ranking.items(), key=lambda item: item[1], reverse=True))

with open("lol_tourney_rankings.json", "w") as fp:
    json.dump(rankings , fp)

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  4827
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  4827
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  4827
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is s

In [ ]:
#rankings

{'110733838935136200': {'109981650516317055': 15,
  '110534741480867943': 15,
  '110733870612047344': 15,
  '107580497041999074': 15,
  '109783149221216052': 15,
  '110534732558665825': 15,
  '109981647134921596': 15,
  '110733864366329081': 15,
  '110534696811298545': 15,
  '110733889090164854': 15,
  '110534728305628996': 15,
  '110534714934488523': 15,
  '107580483738977500': 15,
  '110534700622694005': 15,
  '110733881542862348': 15,
  '110733877357854829': 15},
 '110428723804419399': {'105521293297166647': 7,
  '107067065368380132': 7,
  '109671413728531527': 7,
  '105521706535388095': 7,
  '108078422885368642': 7,
  '105521637552587612': 7,
  '105521674055625312': 7,
  '102787200149928963': 7},
 '110424377524465827': {'107179740624168899': 7,
  '105520822049210915': 7,
  '109539675734129088': 7,
  '105520796734358045': 7,
  '105520786527322647': 7,
  '110424394375218208': 7,
  '110424397251619481': 7,
  '107128093558481418': 7},
 '110349992504762921': {'107564167676290839': 9,
  

In [ ]:
#different output format

out = {id:{} for id in rankings.keys()}
for id in tourney_df.index:
  for teamid in rankings[id]:
    out[id][teamid] = {}
    if teamid in mp.keys():
      out[id][teamid]['name'] = mp[teamid]
    else:
      out[id][teamid]['name'] = teamid
    out[id][teamid]['winrate'] = rankings[id][teamid]/(2*(len(rankings[id])-1))
    out[id][teamid]['winfrac'] = str(rankings[id][teamid]) +' / '+ str(2*(len(rankings[id])-1))

out

with open("lol_tourney_ranks.json", "w") as fp:
    json.dump(out , fp)

In [ ]:
#t_out = {mp[teamid]:{'id':0, 'score':0, 'frac':0} for teamid in global_ranks_sorted.keys()}
#for teamid in global_ranks_sorted.keys():
#  t_out[mp[teamid]]['id'] = teamid
#  t_out[mp[teamid]]['score'] = global_ranks_sorted[teamid]/(2*len(global_ranks_sorted)-1)
#  t_out[mp[teamid]]['winfrac'] = str(global_ranks_sorted[teamid])+' / '+str((2*len(global_ranks_sorted)-1))
#  #g_out[teamid]['acronym'] = team_df['acronym'][teamid]
#t_out

##Global

In [ ]:
recent_df = df[df['gamedate'].dt.year >= 2022]

In [ ]:
global_teams = list(pd.concat([recent_df['finalteam1id'], recent_df['finalteam2id']]).unique())
globalteams = [team for team in global_teams if team in team_df['team_id'].values]
#globalteams

In [ ]:
#team_df['team_id'].values

In [ ]:
#play global tournament
res = play_tourney(globalteams, df, clf)
team_df = pd.read_json("esports-data/teams.json", dtype={'team_id':'str'})
team_df.set_index('team_id')
mp = {team_df['team_id'][i]:team_df['name'][i] for i in team_df.index}

global_ranks = {mp[key]:res[key] for key in res.keys()}
global_ranks

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  9087
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  9087
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
dflen:  9087
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is s

{'Kwangdong Freecs': 111,
 'DRX': 149,
 'Dplus Kia': 207,
 'TSM': 101,
 'T1': 210,
 'Dignitas': 111,
 'Immortals Progressive': 87,
 'Golden Guardians': 189,
 'Evil Geniuses LG': 180,
 'Gen.G': 218,
 'SK Gaming': 152,
 'Miners': 26,
 'paiN Gaming': 119,
 'kt Rolster': 212,
 'EXCEL': 161,
 'Hanwha Life Esports': 202,
 'Misfits Gaming': 125,
 'MAD Lions': 178,
 'FlyQuest': 157,
 'Liiv SANDBOX': 170,
 'Cloud9': 196,
 'KaBuM! Esports': 18,
 'Team Aze': 51,
 'Isurus': 85,
 'CLG': 137,
 'Estral Esports': 100,
 'Frank Esports': 151,
 'Movistar R7': 121,
 'Deep Cross Gaming': 144,
 'OKSavingsBank BRION': 131,
 'LOUD': 145,
 'PSG Talon': 188,
 'All Knights': 23,
 'AXIZ': 46,
 'Crest Gaming Act': 41,
 'NongShim REDFORCE': 128,
 'Beyond Gaming': 167,
 '100 Thieves': 126,
 'Team Liquid Honda': 192,
 'V3 Esports': 18,
 'Sengoku Gaming': 165,
 'Fukuoka SoftBank HAWKS gaming': 134,
 'Burning Core': 55,
 'CTBC Flying Oyster': 172,
 'J Team': 148,
 'INFINITY': 93,
 'Vivo Keyd Stars': 103,
 'RED Kalunga'

In [ ]:
#len(global_ranks)

112

In [ ]:
global_ranks_sorted = dict(sorted(res.items(), key=lambda item: item[1], reverse=True))
#global_ranks_sorted

In [ ]:
#formatting output
g_out = {mp[teamid]:{'id':0, 'score':0, 'frac':0} for teamid in global_ranks_sorted.keys()}
for teamid in global_ranks_sorted.keys():
  g_out[mp[teamid]]['id'] = teamid
  g_out[mp[teamid]]['score'] = global_ranks_sorted[teamid]/(2*len(global_ranks_sorted)-1)
  g_out[mp[teamid]]['winfrac'] = str(global_ranks_sorted[teamid])+' / '+str((2*len(global_ranks_sorted)-1))
  #g_out[teamid]['acronym'] = team_df['acronym'][teamid]
g_out

{'Beijing JDG Intel Esports Club': {'id': '99566404852189289',
  'score': 0.9910313901345291,
  'frac': 0,
  'winfrac': '221 / 223'},
 'Bilibili Gaming Pingan Bank': {'id': '99566404853854212',
  'score': 0.9865470852017937,
  'frac': 0,
  'winfrac': '220 / 223'},
 'Gen.G': {'id': '100205573495116443',
  'score': 0.9775784753363229,
  'frac': 0,
  'winfrac': '218 / 223'},
 'WeiboGaming FAW AUDI': {'id': '99566404853058754',
  'score': 0.968609865470852,
  'frac': 0,
  'winfrac': '216 / 223'},
 'Suzhou LNG Esports': {'id': '99566404850008779',
  'score': 0.9596412556053812,
  'frac': 0,
  'winfrac': '214 / 223'},
 'kt Rolster': {'id': '99566404579461230',
  'score': 0.9506726457399103,
  'frac': 0,
  'winfrac': '212 / 223'},
 'T1': {'id': '98767991853197861',
  'score': 0.9417040358744395,
  'frac': 0,
  'winfrac': '210 / 223'},
 'Dplus Kia': {'id': '100725845018863243',
  'score': 0.9282511210762332,
  'frac': 0,
  'winfrac': '207 / 223'},
 'G2 Esports': {'id': '98767991926151025',
  '

In [ ]:
#more formatting
gout = []
for teamid in global_ranks_sorted.keys():
  team_object = {}
  team_object['name'] = mp[teamid]
  team_object['id'] = teamid
  team_object['score'] = global_ranks_sorted[teamid]/(2*len(global_ranks_sorted)-1)
  team_object['winfrac'] = str(global_ranks_sorted[teamid])+' / '+str((2*len(global_ranks_sorted)-1))
  gout.append(team_object)
gout

[{'name': 'Beijing JDG Intel Esports Club',
  'id': '99566404852189289',
  'score': 0.9910313901345291,
  'winfrac': '221 / 223'},
 {'name': 'Bilibili Gaming Pingan Bank',
  'id': '99566404853854212',
  'score': 0.9865470852017937,
  'winfrac': '220 / 223'},
 {'name': 'Gen.G',
  'id': '100205573495116443',
  'score': 0.9775784753363229,
  'winfrac': '218 / 223'},
 {'name': 'WeiboGaming FAW AUDI',
  'id': '99566404853058754',
  'score': 0.968609865470852,
  'winfrac': '216 / 223'},
 {'name': 'Suzhou LNG Esports',
  'id': '99566404850008779',
  'score': 0.9596412556053812,
  'winfrac': '214 / 223'},
 {'name': 'kt Rolster',
  'id': '99566404579461230',
  'score': 0.9506726457399103,
  'winfrac': '212 / 223'},
 {'name': 'T1',
  'id': '98767991853197861',
  'score': 0.9417040358744395,
  'winfrac': '210 / 223'},
 {'name': 'Dplus Kia',
  'id': '100725845018863243',
  'score': 0.9282511210762332,
  'winfrac': '207 / 223'},
 {'name': 'G2 Esports',
  'id': '98767991926151025',
  'score': 0.9147

In [ ]:
with open("lol_global_ranks.json", "w") as fp:
  json.dump(gout , fp)

In [ ]:
#team_df.head()

In [ ]:
#test = df[(df['finalteam1id']=='102235771678061291')|(df['finalteam2id']=='102235771678061291')]
#test[['gamedate', 'leaguename', 'finalteam1id', 'finalteam2id', 'leaguevswinrate1', 'strength20_1', 'strength90_1', 'strength365_1','strength20_2', 'strength90_2', 'strength365_2', 'pex1', 'pex2']]

In [ ]:
#stages = tourney_df['stages'].loc['110733838935136200']

#stages
#for stage in stages:
#  print(stage['name'])

In [ ]:
#df[df['league2']=='msi']